# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities=pd.read_csv("../WeatherPY/weather_data.csv")
cities.head()

,Unnamed: 0,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Temp,Wind Speed
0,0,300,ushuaia,75,AR,1611953364,75,-54.8000,-68.3000,5.00,11.32
1,1,721,harindanga,20,IN,1611953364,82,22.0383,88.3214,17.00,2.06
2,2,801,coihaique,20,CL,1611953364,39,-45.5752,-72.0662,17.00,5.14
3,3,800,kruisfontein,1,ZA,1611953200,80,-34.0033,24.7314,17.22,0.45
4,4,803,albany,64,US,1611952789,43,42.6001,-73.9662,-8.84,5.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
gmaps.configure(api_key=g_key)

In [27]:
locations= cities[["Lat", "Long"]]

 # Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig =gmaps(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
# this works getting location markers to show
#fig

TypeError: 'module' object is not callable

In [32]:
 #adding the heat map
locations= cities[["Lat", "Long"]]

rating=cities['Humidity'].astype(float)

# plot heatmap
fig = gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations,weights=rating,dissipating=False,max_intensity=7,
                                point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
